In [1]:
import pandas as pd
from sklearn import datasets
import math
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import pydotplus

In [3]:
iris=datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
df=pd.DataFrame(iris.data)
df.columns=["sl","sw","pl","pw"]
df.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
def Label(val,*par):
    if val<par[0]:
        return "1"
    if val<par[1]:
        return "2"
    if val<par[2]:
        return "3"
    return "4"

def toLabel(df,feature):
    second=df[feature].mean()
    first=(df[feature].min()+second)/2
    third=(df[feature].max()+second)/2
    return df[feature].apply(Label,args=(first,second,third))
    

In [6]:
df["sepal_length"]=toLabel(df,"sl")
df["sepal_width"]=toLabel(df,"sw")
df["petal_length"]=toLabel(df,"pl")
df["petal_width"]=toLabel(df,"pw")
df.head()

,sl,sw,pl,pw,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2,2,3,1,1
1,4.9,3.0,1.4,0.2,1,2,1,1
2,4.7,3.2,1.3,0.2,1,3,1,1
3,4.6,3.1,1.5,0.2,1,3,1,1
4,5.0,3.6,1.4,0.2,1,3,1,1


In [7]:
df.drop(["sl","sw","pl","pw"],axis=1,inplace=True)

In [8]:
print(df.head())
set(df["sepal_length"])

  sepal_length sepal_width petal_length petal_width
0            2           3            1           1
1            1           2            1           1
2            1           3            1           1
3            1           3            1           1
4            1           3            1           1


{'1', '2', '3', '4'}

In [9]:
y = pd.DataFrame(iris.target)
unused_features = df.columns
types=iris.target_names

In [10]:
X_train=df.values
Y_train=y.values
clf=DecisionTreeClassifier()
clf.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
dot_data=export_graphviz(clf,out_file=None,filled=True,rounded=True,special_characters=True,feature_names=iris.feature_names,class_names=iris.target_names)
graph=pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("iris2.pdf")

True

In [12]:
def build_tree(df, y,features,level,types):
    #base case
    # 1. unused is empty
    if len(features)==0:
        print("Level",level)
        #finding the diff values y can take
        a=set(y[0])
        #printing count of all output classes
        for ele in a:
            print("Count of",types[ele],"=",y[y[0]==ele].shape[0])
        #printing entropy 
        k=entropy(df,y)
        print("Current Entropy is =",k)
        print("All features are finished")
        print()
        return
        
    
    # 2. y contains only one distinct value
    
    if len(set(y[0]))==1:
        print("Level",level)
        print("Count of",types[set(y[0]).pop()],"=",y.shape[0])
        print("Current entropy is =",0.0)
        print("Reached Leaf Node")
        print()
        return
    
    #findind the feature for which we get best gain ratio
    max_gain=-100000
    best_feature=""
    for f in features:
        #find gain ratio for this feature
        f_gain=gainR(df,y,f)
        #comparing it with max_gain
        if f_gain>max_gain:
            max_gain=f_gain
            best_feature=f
            
    #printing the information of this node       
    print("Level",level)
    print("Count of",types[0],"is",y[y[0]==0].shape[0])
    print("Count of",types[1],"is",y[y[0]==1].shape[0])
    print("Count of",types[2],"is",y[y[0]==2].shape[0])
    print("Current Entropy is=",entropy(df,y))
    print("Splitting on feature",best_feature,"with gain ratio",max_gain)
    print()
    
    #findind set of values best_feature can take
    f_set=set(df[best_feature])
    
    #deleting the best_feature from features list
    b=np.array([best_feature])
    features=np.setdiff1d(features,b)
    
    #dividing df[best_feature] into its classes and calling recursive calls on them
    for ele in f_set:
        new_df=df[df[best_feature]==ele]
        new_y=y[df[best_feature]==ele]
        build_tree(new_df,new_y,features,level+1,types)

In [13]:
def gainR(df,y,feature):
    #calculating initial entropy(of the whole node)
    ini_entropy=entropy(df,y)
    
    #finding the different values of the given feature in df 
    fset=set(df[feature])
    #finding number of total data points in df
    D=y.shape[0]
    
    #Now we will find the entropy after the split due to this feature
    split_entropy=0
    
    sizes=[]#to store the number of data points in each class
    
    #dividing df[feature] on the basis of output classes and finding entropy for them
    for ele in fset:
        new_df=df[df[feature]==ele]
        new_y=y[df[feature]==ele]
        class_size=new_y.shape[0]
        sizes.append(class_size)
        #finding entropy for this class and adding it to  the total split entropy
        split_entropy+=(class_size/D)*entropy(new_df,new_y)
    #calculating information gain for the split
    info_gain=ini_entropy-split_entropy
    #finding split ratio
    split_ratio=0
    for k in sizes:
        split_ratio+=(-k/D)*math.log(k/D)
    #calculating gain ratio
    gain_ratio=info_gain/split_ratio
    return gain_ratio


In [14]:
def entropy(df,y):
    #finding the different classes in y
    Y_set=set(y[0])
    D=y.shape[0]
    ent=0
    for ele in Y_set:
        #finding size of each class
        size=y[y[0]==ele].shape[0]
        #calculating the prob of this class
        prob=size/D
        ent+=-prob*math.log(prob)
    return ent


In [15]:
build_tree(df, y, unused_features,0,types)

Level 0
Count of setosa is 50
Count of versicolor is 50
Count of virginica is 50
Current Entropy is= 1.0986122886681096
Splitting on feature petal_width with gain ratio 0.6996382036222091

Level 1
Count of virginica = 34
Current entropy is = 0.0
Reached Leaf Node

Level 1
Count of setosa is 0
Count of versicolor is 40
Count of virginica is 16
Current Entropy is= 0.5982695885852573
Splitting on feature petal_length with gain ratio 0.4334099495621066

Level 2
Count of virginica = 8
Current entropy is = 0.0
Reached Leaf Node

Level 2
Count of setosa is 0
Count of versicolor is 39
Count of virginica is 8
Current Entropy is= 0.45622342016761397
Splitting on feature sepal_length with gain ratio 0.12674503775809332

Level 3
Count of versicolor = 2
Current entropy is = 0.0
Reached Leaf Node

Level 3
Count of setosa is 0
Count of versicolor is 23
Count of virginica is 7
Current Entropy is= 0.5432727813369008
Splitting on feature sepal_width with gain ratio 0.0709203640514889

Level 4
Count of v

![iris2.JPG](attachment:iris2.JPG)